# Reconstruction (without truncations)
_Author: Lucas Z. Brito_

This notebook reconstructs the Haldane Shastry Hamiltonian. 

### Reconstruction and model parameters

In [76]:
n = 5 # Chain size 
atol = 1e-2 # Absolute error for sanity checks 
output_dir = "../data/hs_data/"

"../data/hs_data/"

In [77]:
using JLD
using DataFrames
using CSV

### Reconstruction

In [78]:
include("./src/hs_tools.jl")

# Make wavefunction and correlation matrix and save to a directory to avoid 
# running calculation again. 
make_wf_cm(n, output_dir) # Can take roughly a minute for n > 10. 


Making wavefunction... Done.
Making correlation matrix... Done.


Saved to ../data/hs_data//cm_wf_n=5.jld


Dict{String, Array} with 2 entries:
  "wf" => ComplexF64[0.0+0.0im, 0.0+0.0im, 0.0+0.0im, 0.129099+0.0937962im, 0.0…
  "cm" => [5.55112e-17 -6.93889e-18; -6.93889e-18 3.46945e-18]

In [79]:
cm_wf = load(output_dir * "cm_wf_n=$n.jld")
k = cm_wf["wf"]
cm = cm_wf["cm"]

# Haldane shastry coefficients
hs = normalize(hs_coeffs(n))
hs

2-element Vector{Float64}:
 0.9341723589627157
 0.35682208977308993

In [80]:
# Make correlation matrix
cm = make_corr_mat(n, k)

printstyled("Correlation matrix: ", bold=true); flush(stdout)
display(cm)

# Diagonalize correlation matrix
eig = eigen(re_cm)

printstyled("Eigenvectors:", bold=true); flush(stdout)
display(eig.vectors)
printstyled("Eigenvalues:", bold=true); flush(stdout)
display(eig.values)

Correlation matrix: 

2×2 Matrix{Float64}:
  5.55112e-17  -6.93889e-18
 -6.93889e-18   3.46945e-18

Eigenvectors:

2×2 Matrix{Float64}:
 -0.129933  -0.991523
 -0.991523   0.129933

Eigenvalues:

2-element Vector{Float64}:
 2.5601488008383387e-18
 5.642044938237311e-17

In [81]:
# Check that conserved quantity and Hamiltonian are in nullspace of CM

nullsp = [normalize(eig.vectors[:, i]) for i in 1:2]
h_proj, h_is_in_nsp = in_nsp(nullsp, hs, atol)
in_nsp([eig.vectors[:, 1]], hs, atol)

conserved_quant = normalize(ones(trunc(Int, n / 2)))
stot_proj, stot_is_in_nsp = in_nsp(nullsp, conserved_quant, atol)

# Project HS coefficient onto nullspace; this will be our "reconstructed 
# Hamiltonian."
coeffs_recon = reduce((a, b) -> a + dot(b, hs) * normalize(b), nullsp; 
					init=zeros(trunc(Int, n / 2)))

if h_is_in_nsp
	printstyled("Hamiltonian is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("Hamiltonian is not in nullspace ×", bold=true, color=:red)
end 

println()

if stot_is_in_nsp
	printstyled("S_tot is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("S_tot is not in nullspace ×", bold=true, color=:red)
end 

println()
println("(atol = $atol)")

Hamiltonian is in nullspace ✓
S_tot is in nullspace ✓
(atol = 0.01)


In [82]:
printstyled("Projections onto nullspace:\n", bold=true)
print("H:\t")
print(h_proj)
println()
print("S_tot:\t")
print(stot_proj)
println()

Projections onto nullspace:
H:	0.9999999999999999
S_tot:	0.9999999999999998


In [83]:
# Save data 
begin
	df = DataFrame(r=1:(trunc(Int, n / 2)), 
				   n=n,
				   coeffs_anal=hs, 
				   coeffs_recon=real.(coeffs_recon))
	fn = CSV.write(output_dir * "recon_n=$n.csv", df)
	printstyled("Written reconstruction data to $fn.", bold=true, color=:green)
end

Written reconstruction data to ../data/hs_data/recon_n=5.csv